### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle
from tqdm import tqdm


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [2]:
users_with_all_recs = [user for user, recs in all_recs.items() 
                       if len(recs) >= 10]
users_who_need_recs = [user for user, recs in all_recs.items() 
                       if len(recs) < 10]

In [3]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [4]:
# create a dataframe similar to reviews, but ranked by rating for each user
sorted_reviews = reviews.sort_values(['user_id', 'rating'], ascending=False)
sorted_reviews.head(200)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
712335,53968,1559547,2,1373287369,2013-07-08 12:42:49,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
712336,53968,2415464,2,1373772560,2013-07-14 03:29:20,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
712334,53967,816711,8,1371972851,2013-06-23 07:34:11,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
711725,53966,338751,10,1362328708,2013-03-03 16:38:28,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
711730,53966,404032,10,1453328244,2016-01-20 22:17:24,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
711732,53966,407887,10,1365945398,2013-04-14 13:16:38,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
711733,53966,421715,10,1362328629,2013-03-03 16:37:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
711734,53966,451279,10,1504145207,2017-08-31 02:06:47,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
711799,53966,1277737,10,1367357104,2013-04-30 21:25:04,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
711811,53966,1327773,10,1388004171,2013-12-25 20:42:51,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [in the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar, and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (and will also be the number of 1's in the movie row within the orginal movie content matrix).

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [5]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
subset_movies = movies.drop(['movie_id', 'movie', 'genre', 'date'], axis=1).values

# Take the dot product to obtain a movie x movie matrix of similarities
dot_prod_movies = subset_movies.dot(subset_movies.T)

In [6]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 31245
assert dot_prod_movies.shape[1] == 31245
assert dot_prod_movies[0, 0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [7]:
dot_prod_movies.shape

(31245, 31245)

In [8]:
sample = dot_prod_movies[:100]

In [9]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [10]:
movie_id = 12
movie_row = movies[movies.movie_id == movie_id].index[0]
movie_row

2

In [11]:
movie_id = 8
# sorted(sample[movie_id], reverse=True)

In [12]:
?np.percentile

In [13]:
threshold = np.percentile(sample[movie_row], 99.9)

In [14]:
(sample[movie_row] >= threshold).sum()

148

In [15]:
31245 * 0.01

312.45

In [16]:
movie_titles = movies.movie
movie_titles.head()

0        Edison Kinetoscopic Record of a Sneeze (1894)
1                  La sortie des usines Lumière (1895)
2                        The Arrival of a Train (1896)
3    The Oxford and Cambridge University Boat Race ...
4                           Le manoir du diable (1896)
Name: movie, dtype: object

In [17]:
dot_prod_movies[movie_row]

array([3, 3, 3, ..., 0, 0, 0])

In [18]:
similar = movies[['movie_id', 'movie']].copy()
similar['sim'] = dot_prod_movies[movie_row]
similar = similar.sort_values('sim', ascending=False)
similar.head()

,movie_id,movie,sim
0,8,Edison Kinetoscopic Record of a Sneeze (1894),3
2,12,The Arrival of a Train (1896),3
12766,392728,Roundhay Garden Scene (1888),3
1,10,La sortie des usines Lumière (1895),3
9584,154152,Annabelle Serpentine Dance (1895),3


In [19]:
similar.movie.values[:10]

array(['Edison Kinetoscopic Record of a Sneeze (1894)',
       'The Arrival of a Train (1896)', 'Roundhay Garden Scene (1888)',
       'La sortie des usines Lumière (1895)',
       'Annabelle Serpentine Dance (1895)',
       'Llandudno Happy Valley and Minstrel Show (1898)',
       'Urban Outlaw (2012)', 'Khaneh siah ast (1963)',
       'Les maîtres fous (1955)', 'Karins ansikte (1984)'], dtype=object)

In [20]:
user = 71

In [21]:
reviewed_movies = set(reviews[reviews.user_id == user].movie_id.dropna())
reviewed_movies

{1234548, 2975590, 4206218}

In [23]:
u = np.array([0, 3, 4, 5])
u[:10]

array([0, 3, 4, 5])

In [32]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title.
    '''
    # find the row of each movie id
    movie_row = movies[movies.movie_id == movie_id].index[0]
    
    similar = movies[['movie_id', 'movie']].copy()
    similar['sim'] = dot_prod_movies[movie_row]
    similar = similar.sort_values('sim', ascending=False)
    # TODO: Randomize in case of tie
    similar_movies = similar
    
    return similar_movies.iloc[:10].movie.values
    
# You made this function in an earlier notebook - using again here    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
   
    return movie_lst


def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    recs = dict()
    
    # For each user
    for user in tqdm(users_who_need_recs):

        # Pull only the reviews the user has seen
        reviewed_movies = list(set(reviews[
            reviews.user_id == user].movie_id.dropna()))
        ten_movies = np.random.choice(reviewed_movies, 10)
        
        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user
        similar = {movie: find_similar_movies(movie) 
                   for movie in ten_movies}
        user_recs = set()
        # Get the most similar for each movie, then the second, etc.
        for level in range(10):
            for movie in ten_movies:
                if level < len(similar[movie]):
                    user_recs.add(similar[movie][level])
                if len(user_recs) >= 10:
                    break
            if len(user_recs) >= 10:
                break
        
        recs[user] = list(user_recs)
    
    return recs

In [33]:
recs = make_recs()

100%|██████████| 1325/1325 [02:13<00:00,  9.95it/s]


### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [34]:
# Explore recommendations
recs

{26: ['La tigre e la neve (2005)',
  'The Wackness (2008)',
  'Gaga: Five Foot Two (2017)',
  'Testosterone (2003)',
  'Peep World (2010)',
  'Jason Becker: Not Dead Yet (2012)',
  'Marley (2012)',
  'Vivir mata (2002)',
  'Falling Up (2009)',
  'What a Girl Wants (2003)'],
 71: ['Little Boy (2015)',
  'Une jeunesse allemande (2015)',
  'Na Wewe (2010)',
  'In Search of Blind Joe Death: The Saga of John Fahey (2013)',
  'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
  'Mein bester Feind (2011)',
  'The Weather Underground (2002)',
  'The Amazing Spider-Man 2 (2014)',
  'Dying to Know: Ram Dass &amp; Timothy Leary (2014)',
  'Xi you ji zhi: Sun Wukong san da Baigu Jing (2016)'],
 108: ['Wicked Blood (2014)',
  'Zvezda (2002)',
  'Sunshine State (2002)',
  'My Last Valentine in Beirut in 3D (2012)',
  'A Clean Exit (2016)',
  'Raaz (2002)',
  'Lust Stories (2018)',
  'Stage Fright (2014)',
  'Haebaragi (2006)',
  'What a Girl Wants (2003)'],
 109: ['Hell Night (1981)',


In [38]:
[user for user, r in recs.items() if len(r) < 10]

[]

In [39]:
[user for user, r in recs.items() if len(r) > 10]

[]

In [40]:
[user for user, r in recs.items() if len(r) == 10]

[26,
 71,
 108,
 109,
 130,
 190,
 208,
 257,
 262,
 268,
 314,
 321,
 341,
 360,
 466,
 511,
 565,
 584,
 590,
 646,
 649,
 693,
 694,
 697,
 730,
 750,
 762,
 822,
 850,
 998,
 1060,
 1083,
 1094,
 1110,
 1154,
 1176,
 1236,
 1247,
 1310,
 1346,
 1351,
 1355,
 1362,
 1364,
 1376,
 1385,
 1475,
 1479,
 1482,
 1541,
 1582,
 1617,
 1757,
 1788,
 1799,
 1822,
 1886,
 1928,
 1953,
 1987,
 1992,
 2014,
 2019,
 2050,
 2174,
 2248,
 2265,
 2306,
 2481,
 2513,
 2524,
 2558,
 2567,
 2616,
 2728,
 2736,
 2779,
 2860,
 2875,
 2932,
 3017,
 3035,
 3112,
 3152,
 3190,
 3202,
 3237,
 3333,
 3427,
 3429,
 3493,
 3536,
 3565,
 3585,
 3613,
 3664,
 3676,
 3689,
 3832,
 3852,
 3889,
 3895,
 3987,
 4028,
 4098,
 4208,
 4242,
 4243,
 4319,
 4340,
 4369,
 4382,
 4480,
 4550,
 4571,
 4592,
 4643,
 4689,
 4711,
 4818,
 4822,
 4842,
 4933,
 4980,
 4988,
 5054,
 5161,
 5272,
 5284,
 5332,
 5367,
 5383,
 5417,
 5456,
 5493,
 5503,
 5646,
 5664,
 5666,
 5706,
 5721,
 5729,
 5740,
 5905,
 5939,
 5981,
 6040,
 60

In [41]:
# Some characteristics of my content based recommendations
users_without_all_recs = [user for user, r in recs.items() if len(r) < 10]
users_with_all_recs = [user for user, r in recs.items() if len(r) == 10]
no_recs = [user for user, r in recs.items() if len(r) == 0]

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 0 users without all 10 recommendations we would have liked to have.
There were 1325 users with all 10 recommendations we would like them to have.
There were 0 users with no recommendations at all!


In [42]:
# Closer look at individual user characteristics - this may help it was from an earlier notebook
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


movies_watched(189)

array([457430])

In [43]:
# More exploring
print("Some of the movie lists for users without any recommendations include:")
# Just one of many questions you could continue exploring...

Some of the movie lists for users without any recommendations include:


### Now What?  

Well, if you were really strict with your criteria for how similar two movies are (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!

In [ ]:
# Cells for exploring

My approach already gives 10 recommendations to all the users. It looks for the most similar movies to each movie the user has watched. It doesn't look for inter-movie ranks. That means, if the user has seen movie1 and movie2, the most similar item to movie1 may still be less similar then the 2nd or 3rd most similar item to movie2. That isn't taken into account with my approach. Another option would be to mix all the similarity measures and choose really those that are closer to any of the watched movies. There is a risk in that approach, however. If a movie has very strong similarity measures to many other movies the recommendation may suffer from a great lack of diversity. So, I think I would keep my approach, by now. I would like to test how good it is, but that may have to wait until the next lesson...